*Note: Sorry for submitting a pdf - I'm not sure why, but my shared Colab notebook rendered the Altair visualizations only in certain web browsers, so this seemed less elegant but safer*

# Assignment 2: Global infrastructure data from the World Bank
Kevin Bishop | CSE 512

This [dataset](https://github.com/ZeningQu/World-Bank-Data-by-Indicators/tree/master/infrastructure) contains infrastructure indicators (e.g. number of fixed telephone subscriptions per 100 people, electrical power consumption per capita) for different countries and aggregate regions for different years through 2017. For simplicity I've focused on a single year as described below. Some wrangling and cleaning was done by Zening Qu.

Specific analysis questions:
1. How does mode of phone communication (mobile vs. landline) vary worldwide?
2. How are access to different modes of communication (such as phone and internet) related?
3. How does method of electricity generation vary worldwide, and is it related to the amount of electricity used?

## Data overview

In [1]:
import pandas as pd
import altair as alt
import numpy as np
alt.data_transformers.enable('data_server')
#alt.data_transformers.enable('default')

from altair import datum
from vega_datasets import data

In [2]:
data = pd.read_csv('https://github.com/ZeningQu/World-Bank-Data-by-Indicators/raw/master/infrastructure/infrastructure.csv')
meta_country = pd.read_csv('https://github.com/ZeningQu/World-Bank-Data-by-Indicators/raw/master/infrastructure/Metadata_Country_API_9_DS2_en_csv_v2_10135164.csv')
meta_indicator = pd.read_csv('https://github.com/ZeningQu/World-Bank-Data-by-Indicators/raw/master/infrastructure/Metadata_Indicator_API_9_DS2_en_csv_v2_10135164.csv')

In [3]:
data.head(3)

,Country Name,Country Code,Year,"Air transport, freight (million ton-km)","Air transport, passengers carried","Air transport, registered carrier departures worldwide","Annual freshwater withdrawals, agriculture (% of total freshwater withdrawal)","Annual freshwater withdrawals, domestic (% of total freshwater withdrawal)","Annual freshwater withdrawals, industry (% of total freshwater withdrawal)","Annual freshwater withdrawals, total (% of internal resources)",...,"Quality of port infrastructure, WEF (1=extremely underdeveloped to 7=well developed and efficient by international standards)",Rail lines (total route-km),"Railways, goods transported (million ton-km)","Railways, passengers carried (million passenger-km)",Renewable internal freshwater resources per capita (cubic meters),"Renewable internal freshwater resources, total (billion cubic meters)",Secure Internet servers,Secure Internet servers (per 1 million people),"Trademark applications, nonresident, by count","Trademark applications, resident, by count"
0,Albania,ALB,1977,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,10702.012217,26.9,0,0.000000,0,0
1,Azerbaijan,AZE,2013,17.92160,1651710.0,15886.0,0.0,0.0,0.0,0.0,...,4.500000,2068.1,7958.3,591.3,0.000000,0.0,158,16.778522,10494,3556
2,Azerbaijan,AZE,2015,686.94149,1803112.0,23781.0,0.0,0.0,0.0,0.0,...,4.259906,2067.8,6210.5,494.7,0.000000,0.0,321,33.266520,9304,2801


Below are some further cleaning steps, specifically using indicator codes instead of complete names for more efficient access of variables. Finally I list all included variables, which revealed that this list is actually a subset of what is described in the original data.

In [4]:
#Rename columns using Indicator Codes
code_dict = dict(meta_indicator.filter(items=['INDICATOR_NAME', 'INDICATOR_CODE']).values)

#Remove periods in Indicator Codes for compatability with Altair
for key in code_dict:
    code_dict[key] = code_dict[key].replace('.','')
    
#print(meta_indicator.filter(items=['INDICATOR_NAME']))

data.rename(columns=code_dict, inplace=True)

#print(list(data.columns))

#[mydict[x] for x in mykeys]

inv_code_dict = {v: k for k, v in code_dict.items()}
print(*[inv_code_dict[x] for x in list(data.columns[3:])], sep='\n')

#shows that I am missing many expected variables!


Air transport, freight (million ton-km)
Air transport, passengers carried
Air transport, registered carrier departures worldwide
Annual freshwater withdrawals, agriculture (% of total freshwater withdrawal)
Annual freshwater withdrawals, domestic (% of total freshwater withdrawal)
Annual freshwater withdrawals, industry (% of total freshwater withdrawal)
Annual freshwater withdrawals, total (% of internal resources)
Annual freshwater withdrawals, total (billion cubic meters)
Container port traffic (TEU: 20 foot equivalent units)
Electric power consumption (kWh per capita)
Electric power transmission and distribution losses (% of output)
Electricity production from coal sources (% of total)
Electricity production from hydroelectric sources (% of total)
Electricity production from natural gas sources (% of total)
Electricity production from nuclear sources (% of total)
Electricity production from oil sources (% of total)
Fixed broadband subscriptions
Fixed broadband subscriptions (per 10

This analysis will use just a single year to compare different countries. We start with 2017 as it's the most recent one in the dataset.

In [5]:
data_year = data[data['Year'] == 2017]
print(data_year.shape)
data_year.head(3)

(261, 50)


,Country Name,Country Code,Year,ISAIRGOODMTK1,ISAIRPSGR,ISAIRDPRT,ERH2OFWAGZS,ERH2OFWDMZS,ERH2OFWINZS,ERH2OFWTLZS,...,IQWEFPORTXQ,ISRRSTOTLKM,ISRRSGOODMTK6,ISRRSPASGKM,ERH2OINTRPC,ERH2OINTRK3,ITNETSECR,ITNETSECRP6,IPTMKNRCT,IPTMKRSCT
27,Isle of Man,IMN,2017,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,380,4508.405804,0,0
53,"Korea, Dem. People’s Rep.",PRK,2017,0.251493,103560.0,460.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0.039230,0,0
95,Gabon,GAB,2017,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,58,28.640038,0,0


To get an overview of the data, let's view it as a heatmap. This requires some reformatting first:

In [6]:
#Heatmap of indicators vs. name
data_year_heat = data_year.drop(columns=['Country Code', 'Year']).melt(id_vars='Country Name')
data_year_heat.insert(3,'Indicator Name',data_year_heat['variable'].map(inv_code_dict),True)

data_year_heat.head()


,Country Name,variable,value,Indicator Name
0,Isle of Man,ISAIRGOODMTK1,0.000000,"Air transport, freight (million ton-km)"
1,"Korea, Dem. People’s Rep.",ISAIRGOODMTK1,0.251493,"Air transport, freight (million ton-km)"
2,Gabon,ISAIRGOODMTK1,0.000000,"Air transport, freight (million ton-km)"
3,Togo,ISAIRGOODMTK1,8.565828,"Air transport, freight (million ton-km)"
4,Switzerland,ISAIRGOODMTK1,1581.353512,"Air transport, freight (million ton-km)"


In [7]:
#based on http://www.shaunadkins.com/2018/05/11/quick-heatmap-in-altair.html

alt.Chart(data_year_heat).transform_joinaggregate(
    max_val='max(value)',
    groupby=['variable']
).transform_calculate(
    #no log version comfirms all squares have some value, so log is both providing log scale and making true zeros white
    norm_val='log(datum.value / datum.max_val)'
).mark_rect().encode(
    alt.X('variable:N'),
    alt.Y('Country Name:N'),
    alt.Color('norm_val:Q'),
    alt.Tooltip(['value:Q','Country Name:N','Indicator Name']),
)

alt.Chart(...)

The above heatmap shows normalized indicator values for each country (normalized relative to the highest value for a particular indicator). Normalized values are colored on a log scale, which incidentally renders zeros as missing values. There are no missing values in the raw data, missing indicators are encoded as zero, so this makes some sense.

While there are some shortcomings to this approach, it allows us to efficiently look for gaps in the data. There are some gaps. The most obvious pattern is that many small countries do not have values for certain indicators. This makes sense in some cases - a small country may simply not export any ITC services. In other cases, this does not make sense (for instance, the percentage of people using the internet in Vietnam is certainly >0%). These cases represent gaps in the data.

There are also many fewer indicators than expected based on the list above. Is this because it's too recent of a year? Let's try 2010 instead.

In [8]:
data_year = data[data['Year'] == 2010]

data_year_heat = data_year.drop(columns=['Country Code', 'Year']).melt(id_vars='Country Name')
data_year_heat.insert(3,'Indicator Name',data_year_heat['variable'].map(inv_code_dict),True)

#based on http://www.shaunadkins.com/2018/05/11/quick-heatmap-in-altair.html

alt.Chart(data_year_heat).transform_joinaggregate(
    max_val='max(value)',
    groupby=['variable']
).transform_calculate(
    #no log version comfirms all squares have some value, so log is both providing log scale and making true zeros white
    norm_val='log(datum.value / datum.max_val)'
).mark_rect().encode(
    alt.X('variable:N'),
    alt.Y('Country Name:N'),
    alt.Color('norm_val:Q'),
    alt.Tooltip(['value:Q','Country Name:N','Indicator Name']),
)



alt.Chart(...)

Looks like 2010 is a more complete year, but there are still some gaps which we will keep in mind. Let's proceed with 2010 and try to answer our first analysis question.

## 1. How does mode of phone communication (mobile vs. landline) vary worldwide?

It might make sense for there to be a relationship between prevalence of landlines and cell phones. One might expect that countries with many cell phones don't need as many landlines. Let's see if that's true by plotting mobile phone subscritions vs. fixed phone subscriptions:



In [9]:
points = alt.Chart(data_year).mark_circle().encode(
    alt.X('ITCELSETSP2:Q', title=inv_code_dict['ITCELSETSP2']),
    alt.Y('ITMLTMAINP2:Q', title=inv_code_dict['ITMLTMAINP2']),
    alt.Tooltip(['Country Name', 'ITCELSETSP2:Q', 'ITMLTMAINP2:Q'])
)

# annotations from: https://stackoverflow.com/questions/44656141/how-to-do-annotations-with-altair
annotation = alt.Chart(data_year).mark_text(
    align='left',
    baseline='middle',
    fontSize=10,
    dx=7
).encode(
    alt.X('ITCELSETSP2:Q'),
    alt.Y('ITMLTMAINP2:Q'),
    alt.Text('Country Name')
).transform_filter(
    ('datum.ITCELSETSP2 == 0 && datum.ITMLTMAINP2>20')
)

points + annotation

alt.LayerChart(...)

Interesting! There seem to be two main groups of countries. One group with very few landlines but ~20 to 60 cell phones per 100 people. The second group, with more of both types of phones, shows no clear relationship between number of cell phones and landlines. 

As a side note, tooltips indicate that the outlier countries with no cell phones and many landlines are all US territories. At first this might look like something special about US territories, but a quick Google search indicates there is indeed cell phone service in Guam. So more likely these are holes in the data.

There is significant variation in number of cell phones, so perhaps other variables are correlated to this metric. This leads nicely into our second question.

## 2. How are access to different modes of communication (such as phone and internet) related?

Let's explore relationships between a few candidate variables using a scatter plot matrix:
- ITCELSETSP2: Mobile cellular subscriptions (per 100 people)
- ITMLTMAINP2: Fixed telephone subscriptions (per 100 people)
- ITNETBBNDP2: Fixed broadband subscriptions (per 100 people)
- ITNETUSERZS: Individuals using the Internet (% of population)

In [10]:
alt.Chart().mark_point(filled=True, size=15, opacity=0.5).encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).properties(
  width=150,
  height=150
).repeat(
  data=data_year,
  row=['ITCELSETSP2', 'ITMLTMAINP2', 'ITNETBBNDP2', 'ITNETUSERZS'],
  column=['ITNETUSERZS', 'ITNETBBNDP2', 'ITMLTMAINP2', 'ITCELSETSP2']
)

alt.RepeatChart(...)

Unsurprisingly, there is a strong correlation between number of fixed broadband subscriptions (ITNETBBNDP2) and percentage of population using the the internet (ITNETUSERZS). However, it does look like there are many countries with 0-20% of the population using the internet despite having very few broadband subscriptions. Maybe people in these countries are largely accessing the internet on their phones - do these countries have higher rates of mobile phone ownership to account for this? We can find out by zooming in on this plot and color coding each point by the mobile phone subscription rate:

In [11]:
alt.Chart(data_year).mark_circle(size=15, opacity=0.5).encode(
    alt.X('ITNETBBNDP2:Q', title=inv_code_dict['ITNETBBNDP2']),
    alt.Y('ITNETUSERZS:Q', title=inv_code_dict['ITNETUSERZS']),
    alt.Tooltip(['Country Name', 'ITNETBBNDP2', 'ITNETUSERZS']),
    alt.Color('ITCELSETSP2', title=inv_code_dict['ITCELSETSP2'])
)

alt.Chart(...)

There are enough overlapping points in this region that a simple color encoding still makes it hard to see exactly what's going on. Let's try a different approach: let's use small multiples, with a separate plot for each quintile in terms of cell phone subscriptions per capita.

Since the assignment of quintiles is impacted by the number of observations in each bin, any missing values encoded as zeros will throw off the assignment. In this case, a value of zero mobile phone users, broadband subscriptions, or people using the internet doesn't make much sense, so we will filter out all zero values in these plots.

In [12]:
base = alt.Chart(data_year).mark_circle(size=15, opacity=0.5).transform_filter(
    'datum.ITNETBBNDP2 != 0 && datum.ITNETUSERZS != 0 && datum.ITCELSETSP2 != 0'
).transform_window(
    percent_rank='percent_rank()',
    sort=[alt.SortField('ITCELSETSP2', order='descending')]
).properties(
  width=150,
  height=150
).encode(
    alt.Tooltip(['Country Name:N', 'ITNETBBNDP2:Q', 'ITNETUSERZS:Q', 'ITCELSETSP2:Q', 'percent_rank:Q']),
)

quintile_names = ['1st', '2nd', '3rd', '4th', '5th']
quintile_names.reverse()

chart = alt.hconcat().configure_axis(
    titleFontSize = 11
    )
for rank in [0.2, 0.4, 0.6, 0.8, 1.0]:
    
    #chart title
    if rank == 0.2:
        more_text = ' (most cell phones)'
    elif rank == 1.0:
        more_text = ' (fewest cell phones)'
    else:
        more_text = ''
        
    #x axis
    if rank == 0.6:
        x_label = ['Fixed broadband subscriptions', '(per 100 people)']
                   #inv_code_dict['ITNETBBNDP2']
    else:
        x_label = ''
    
    #y axis
    if rank == 0.2:
        y_label = ['Individuals using the internet', '(% of population)']
    else:
        y_label = ''
    
    chart |= base.transform_filter('datum.percent_rank < ' + str(rank) + 
                                   ' && datum.percent_rank >= ' + str(rank-0.2)
        ).properties(
            title=(quintile_names[int(rank*5-1)] + ' quintile' + more_text)
        ).encode(
            alt.X('ITNETBBNDP2:Q', title=x_label, scale=alt.Scale(domain=[0, 70])),
            alt.Y('ITNETUSERZS:Q', title=y_label, scale=alt.Scale(domain=[0, 100])),
        )
chart


alt.HConcatChart(...)

Breaking the data into quintiles based on cell phone subscriptions shows that in general, countries with lower rates of mobile phone usage have lower internet usage and broadbrand subscriptions rates. However, there is still significant variability in these variables with respect to mobile phone usage. Countries with high internet access rates compared to broadband subscriptions don't consistently have particularly high rates of cell phone usage, so folks using the internet in those places may be accessing the internet in other ways (internet cafes, public libraries, etc.). In other words, there doesn't seem to be clear evidence supporting our hypothesis that higher cell phone ownership is associated with the combination of high internet access and few broadband subscriptions.

## 3. How does method of electricity generation vary worldwide, and is it related to the amount of electricity used?

Now let's look at how energy is generated in different countries. This dataset includes metrics for oil, nuclear, natural gas, hydroelectric, and coal generation, so a stacked bar chart seems like a good way to look at the breakdown for each country. To just get an overview of the data, we don't need to know which country is which, so we will make the bars compressed and remove the country names. We will also filter out all zero values, which will omit any countries which list 0 generation for all five categories (more likely missing data points than countries getting energy entirely from sources outside these categories).

In [13]:
#set colors
color_domain = ['EGELCPETRZS', 'EGELCNUCLZS', 'EGELCNGASZS', 'EGELCHYROZS', 'EGELCCOALZS']    
real_labels =  ['Oil',         'Nuclear',      'Natural gas','Hydroelectric','Coal']

color_domain = color_domain + real_labels

color_range =  ['8E442B',      'DBC048',      'F57501',      '497FF5',      '331A09']
color_range = color_range + color_range

source_order = ['EGELCCOALZS', 'EGELCPETRZS', 'EGELCNGASZS', 'EGELCHYROZS', 'EGELCNUCLZS']

#extract relevant variables
alt.Chart(data_year_heat).transform_filter(
    (
    (datum.variable == 'EGELCPETRZS') |
    (datum.variable == 'EGELCNUCLZS') |
    (datum.variable == 'EGELCNGASZS') |
    (datum.variable == 'EGELCHYROZS') |
    (datum.variable == 'EGELCCOALZS')) &
    (datum.value != 0)

).mark_bar().encode(
    alt.X('Country Name:N', axis = alt.Axis(labels=False, ticks=False, title='Country'),
          sort=alt.EncodingSortField(field="value", op="max", order='ascending')),
    alt.Y('value:Q', title='Percent of total generation', scale=alt.Scale(domain=[0, 100])),
    alt.Fill('variable:N',
            scale=alt.Scale(domain=color_domain, range=color_range),
            legend=alt.Legend(values=real_labels)
            ),
    alt.Stroke('variable:N', scale=alt.Scale(domain=color_domain, range=color_range)),
    alt.Tooltip(['Country Name:N', 'value:Q', 'variable:N']),
).configure_scale(
    bandPaddingInner=0
).properties(
    width = 750
)

alt.Chart(...)

There is a lot of information here, but we can still notice some trends. Most countries have a mix of different energy sources, though a few rely heavily on a single source. Nuclear energy seems to be the least common, which is probably not surprising. We also see that most countries get most of their energy from these five sources, but a few countries have large amounts of energy generation unaccounted for. It could be that these countries get lots of energy from sources not included here (such as wind or solar), or these could be gaps in the data. There's no way to know which based on this dataset.

Let's now see if there's any relationship between a country's electricity consumption (per capita) and what its breakdown of generation sources is. We can make a scatter plot with energy consumption on the x-axis, and the percentage generation from each source on the y-axis. We'll also show the x-axis in log scale, since there is a large range of consumption values.

In [14]:
#set colors
color_domain = ['EGELCPETRZS', 'EGELCNUCLZS', 'EGELCNGASZS', 'EGELCHYROZS', 'EGELCCOALZS']    
real_labels =  ['Oil',         'Nuclear',      'Natural gas','Hydroelectric','Coal']

color_domain = color_domain + real_labels

color_range =  ['8E442B',      'DBC048',      'F57501',      '497FF5',      '331A09']
color_range = color_range + color_range

source_order = ['EGELCCOALZS', 'EGELCPETRZS', 'EGELCNGASZS', 'EGELCHYROZS', 'EGELCNUCLZS']

#create base chart
base = alt.Chart(data_year).encode(
    alt.X('EGUSEELECKHPC:Q', scale=alt.Scale(type='log', domain=[10,60000]), title='Electric power consumption (kWh per capita)'),
    alt.Y(scale=alt.Scale(domain=[0, 100])),
    alt.Tooltip(['Country Name:N']),
).properties(
    width = 750
)

#layer charts
charts_to_layer = [0, 0, 0, 0, 0]

for n in [0, 1, 2, 3, 4]:
    charts_to_layer[n] = base.mark_circle(color=color_range[n]).encode(
        alt.Y(source_order[n], title='Percent of total generation'),
    )
charts_to_layer[0] + charts_to_layer[1] + charts_to_layer[2] + charts_to_layer[3] + charts_to_layer[4]


alt.LayerChart(...)

There doesn't seem to be an obvious connection between power consumption and generation makeup. However, just to be sure, let's break each source into a separate plot and display the data as five small multiples.

In [15]:
#create base chart
base = alt.Chart(data_year).encode(

    alt.Y(scale=alt.Scale(domain=[0, 100])),
    alt.Tooltip(['Country Name:N']),
).properties(
    width = 150,
    height = 150
)

#concatanate charts
charts_to_concat = [0, 0, 0, 0, 0]

for n in [0, 1, 2, 3, 4]:
    if n == 0:
        y_title = 'Percent of total generation'
    else:
        y_title = ''
        
    if n == 2:
        x_title = ['Electric power consumption', '(kWh per capita)']
    else:
        x_title = ''
    charts_to_concat[n] = base.mark_circle(color=color_range[n]).properties(
            title=real_labels[n]
        ).encode(
            alt.Y(source_order[n], title=y_title),
            alt.X('EGUSEELECKHPC:Q',
              scale=alt.Scale(type='log', domain=[10,60000]),
              title=x_title),
    )

charts_to_concat[0] | charts_to_concat[1] | charts_to_concat[2] | charts_to_concat[3] | charts_to_concat[4]


alt.HConcatChart(...)

From the small multiples, it looks like countries favoring coal tend to use more electricity per capita overall, but otherwise there still aren't any clear trends to be seen. It seems like other factors (maybe things like geography, technological advancement, or industrial history) play more of a role in determining how a country gets its energy than the amount of energy used.